#### Convert from RGB to Lab, get 32 unique triplets of L, a, b

In [ ]:
lab = rgb2lab(out1)

print lab.shape

I tried two methods that return the same result:

1) get H, L, combine into 2D array, sort unique rows

2) sort H, with index returned then sort L with index

In [ ]:
width, height, dump = np.shape(out2)
print  width, height, dump

In [ ]:
# method 1
# extract lightness and hue, combine them into a 2D array

# extract
from skimage.color import rgb2lab, lab2lch, lch2lab, lab2rgb
lab = rgb2lab(out2)
lch = lab2lch(lab)
        
lab = np.asarray(lab)
lch = np.asarray(lch)


# reshape
pixels_lab = np.reshape(lab, (width*height, -1))
l1, a, b = np.split(pixels_lab, 3, axis=-1)
        
pixels_lch = np.reshape(lch, (width*height, -1))
l2, c, h = np.split(pixels_lch, 3, axis=-1)
 
# flatten    
import itertools
lM = list(itertools.chain.from_iterable(l2))
hM = list(itertools.chain.from_iterable(h))

# zip together to make 2D numpy array
lhM = np.asarray(zip(hM,lM))


In [ ]:
# Sorting unique rows
# Joe Kington's answer on Stackoverflow: http://stackoverflow.com/a/16971224
def unique_rows(data):
    uniq = np.unique(data.view(data.dtype.descr * data.shape[1]))
    return uniq.view(data.dtype).reshape(-1, data.shape[1])

In [ ]:
uniqLM = unique_rows(lhM)
print np.shape(uniqLM)

In [ ]:
# method 2
# sorting both lightness and hue by hue separately

from skimage.color import rgb2lab, lab2lch, lch2lab, lab2rgb
lab = rgb2lab(out2)
lch = lab2lch(lab)
        
lab = np.asarray(lab)
lch = np.asarray(lch)


pixels_lab = np.reshape(lab, (width*height, -1))
l1, a, b = np.split(pixels_lab, 3, axis=-1)
        
pixels_lch = np.reshape(lch, (width*height, -1))
l2, c, h = np.split(pixels_lch, 3, axis=-1)
                
huniq, unidx = np.unique(h, return_index=True)
luniq = l2[unidx]
cuniq = c[unidx]

# flatten luniq, cuniq
import itertools 
luniqM = list(itertools.chain.from_iterable(luniq))
cuniqM = list(itertools.chain.from_iterable(cuniq))

In [ ]:
# compare output of two methods
lhM2 = np.asarray(zip(huniq,luniqM))
print 'method 2'
print '         H            L' 
print lhM2[:4]
print lhM2[-4:]

print 'method 1'
print '         H            L' 
print uniqLM[:4]
print uniqLM[-4:]

In [ ]:
np.shape(uniqLM)

#### Import a function to plot colored lines in the final plot using the colormap created above


From David Sanders:
https://github.com/dpsanders/matplotlib-examples/blob/master/colorline.ipynb

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm


# Data manipulation:

def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments


# Interface to LineCollection:

def colorline(x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=3, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    
    return lc
        
    
def clear_frame(ax=None): 
    # Taken from a post by Tony S Yu
    if ax is None: 
        ax = plt.gca() 
    ax.xaxis.set_visible(False) 
    ax.yaxis.set_visible(False) 
    for spine in ax.spines.itervalues(): 
        spine.set_visible(False) 

#### Make final plot of the sorted hue, H versus lightness, L, colored by L

In [ ]:
# To color by L, it has to be normalized to [0 1]
luniqM_n=(luniqM-min(luniqM))/(max(luniqM)-min(luniqM))

fig = plt.figure(figsize=(8,4))
plt.xticks(np.arange(0, 2.25*np.pi,0.25*np.pi),[0.,   45.,   90.,  135.,  180.,  225.,  270.,  315., 360.]) 
# Hue as 0-360 angle

ax1 = fig.add_subplot(111)

# ax1.scatter(huniq, luniq)
colorline(huniq,luniqM, luniqM_n, linewidth=4,cmap='gray')
ax1.set_xlim(0, 2.25*np.pi)
ax1.set_ylim(-5, 105)
ax1.text(5, 95, 'H vs. L - colored by L', va='top')
plt.show()

In [ ]:
cmaps = ['Blues', 'BuGn', 'BuPu','GnBu', 'Greens', 'Greys', 'Oranges', 'OrRd','PuBu', 'PuBuGn', 'PuRd',
         'Purples', 'RdPu','Reds', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd','afmhot', 'autumn', 'bone', 'cool',
         'copper', 'gist_heat', 'gray', 'hot','pink', 'spring', 'summer', 'winter','BrBG', 'bwr', 'coolwarm',
         'PiYG', 'PRGn', 'PuOr', 'RdBu', 'RdGy', 'RdYlBu', 'RdYlGn', 'Spectral','seismic', 'Accent', 'Dark2',
         'Paired', 'Pastel1','Pastel2', 'Set1', 'Set2', 'Set3', 'gist_earth', 'terrain', 'ocean', 'gist_stern',
         'brg', 'CMRmap', 'cubehelix', 'gnuplot', 'gnuplot2', 'gist_ncar', 'nipy_spectral', 'jet', 'rainbow',
         'gist_rainbow', 'hsv', 'flag', 'prism']

Next two cells are a test of syntax

In [ ]:
import skimage.color as skl
import matplotlib.cm as cm
import matplotlib.colors as clr

sz = np.amax(np.shape(uniqLM))
mp=np.arange(sz)
             
my_map = cm.get_cmap('jet') 
normmap = clr.Normalize(0, sz) 
colmap = my_map(normmap(mp))
rm = np.tile(colmap[:,0], (sz,1))
gm = np.tile(colmap[:,1], (sz,1))
bm = np.tile(colmap[:,2], (sz,1))
rgb_mp = np.array(zip(rm, gm ,bm))
rgb_map = np.swapaxes(rgb_mp,1,2) # This is a 256 rows by 256 columns image in which each row is a 
                                      # 256-sample colormap.


lab_map = skl.rgb2lab(rgb_map)
lch_map = skl.lab2lch(lab_map)

print np.shape(lch_map)

In [ ]:
hl_map = np.asarray(zip(lch_map[0,:,2],lch_map[0,:,0]))

print '         H            L' 
print hl_map[:4]
print hl_map[-4:]


In [ ]:
# From my tutorial:
# https://github.com/seg/tutorials/blob/master/1408_Evaluate_and_compare_colormaps/How_to_evaluate_and_compare_colormaps.ipynb

mp=np.arange(np.amax(np.shape(uniqLM))) 
import skimage.color as skl
             import matplotlib.cm as cm
import matplotlib.colors as clr

for m in cmaps:
    sz = np.amax(np.shape(uniqLM))
    mp=np.arange(sz)
             
    my_map = cm.get_cmap('jet') 
    normmap = clr.Normalize(0, sz) 
    colmap = my_map(normmap(mp))
    rm = np.tile(colmap[:,0], (sz,1))
    gm = np.tile(colmap[:,1], (sz,1))
    bm = np.tile(colmap[:,2], (sz,1))
    rgb_mp = np.array(zip(rm, gm ,bm))
    rgb_map = np.swapaxes(rgb_mp,1,2) # This is a 256 rows by 256 columns image in which each row is a 
                                      # 256-sample colormap.


    lab_map = skl.rgb2lab(rgb_map)
    lch_map = skl.lab2lch(lab_map)
    hl_map = np.asarray(zip(lch_map[0,:,2],lch_map[0,:,0]))
    
##  # enter here comparison similar to below

http://numpy-discussion.10968.n7.nabble.com/Minimum-distance-between-2-paths-in-3D-td15677.html